Q-4. Imagine you working as a sale manager now you need to predict the Revenue
and whether that particular revenue is on the weekend or not and find the
Informational_Duration using the Ensemble learning algorithm

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score,roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

In [3]:
df=pd.read_csv('online_shoppers_intention.csv')

In [4]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [5]:
# Copying the dataframe to do feature engineering
df1=df.copy()

In [7]:
# Checking for null values
df1.isnull().sum()

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

In [8]:
df.dtypes

Administrative               int64
Administrative_Duration    float64
Informational                int64
Informational_Duration     float64
ProductRelated               int64
ProductRelated_Duration    float64
BounceRates                float64
ExitRates                  float64
PageValues                 float64
SpecialDay                 float64
Month                       object
OperatingSystems             int64
Browser                      int64
Region                       int64
TrafficType                  int64
VisitorType                 object
Weekend                       bool
Revenue                       bool
dtype: object

In [9]:
# Finding the columns that are not numerical so that we can do encoding for these to train our model
from sklearn.preprocessing import OrdinalEncoder
encoder=OrdinalEncoder()

# Assigning 1 for True and 0 for False values judt like in nominal encoding
for i in df1.columns:
    if df1[i].dtype == 'bool':
        df1[i]=df1[i].map({False:0,True:1})

# Assuming Ordinal relation in columns with more than 2 unique values so doing Ordinal Encoding 
for i in df1.columns:
    if df1[i].dtype == 'object':
        np.array(df1[i]).reshape(-1,1)
        encoded_column = pd.DataFrame(encoder.fit_transform(np.array(df1[i]).reshape(-1,1))).astype(int)
        df1[i]=encoded_column
        
        

In [10]:
df1.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,1,1,1,1,2,0,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,2,1,2,2,0,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,4,1,9,3,2,0,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,2,3,2,2,4,2,0,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,2,3,3,1,4,2,1,0


In [11]:
x=df1.drop(columns=['Revenue'])
y=df1.Revenue

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
RFC=RandomForestClassifier()
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=100)

In [22]:
RFC.fit(x_train,y_train)

RandomForestClassifier()

In [23]:
RFC.score(x_train,y_train)

0.9998841385702699

In [24]:
RFC.score(x_test,y_test)

0.9005136523384698

In [25]:
# Using Grid Search CV to tune our model
from sklearn.model_selection import GridSearchCV
param={ 
    'n_estimators':[5,10,50,100,150],
    'criterion':['gini','entropy'],
    'max_depth':range(1,11),
    'min_samples_leaf':range(1,11)    
}
grid_rf=GridSearchCV(RFC,param_grid=param,cv=10,n_jobs=-1,verbose=1)

In [26]:
grid_rf.fit(x_train,y_train)


Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 11),
                         'min_samples_leaf': range(1, 11),
                         'n_estimators': [5, 10, 50, 100, 150]},
             verbose=1)

In [27]:
grid_rf.best_params_

{'criterion': 'entropy',
 'max_depth': 7,
 'min_samples_leaf': 1,
 'n_estimators': 150}

In [29]:
# Assigning the parameter value we got from the Grid Search CV
best_RFC=RandomForestClassifier(criterion='entropy',max_depth=7,min_samples_leaf=1,n_estimators= 150)

In [33]:
# Fitting the training data in the best tuned model
best_RFC.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=150)

In [34]:
best_RFC.score(x_test,y_test)

0.9048391457150581

In [35]:
best_RFC.score(x_train,y_train)

0.9257328235430425

# As the training and testing data accuracy is pretty close, hence the model is a generalised one.